import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization,Input,ReLU,add,MaxPool1D,GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score,mean_absolute_error
from tensorflow.keras import callbacks,Model,layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=1024
cls_num=5
btch=20
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=MixedShapesSmallTrain0(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num+1):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (100, 1025)
first_test_shape= (2425, 1025)
classes_quantity= 5
tr_lbls=	 {1, 2, 3, 4, 5}
Count_labels= [20 20 20 20 20]
max(train_feature_Altitude)= 5.0
min(train_feature_Altitude)= -2.8941765
first_train_sample=
 [1.        1.1309687 1.1204832 ... 1.0931087 1.1025334 1.1120469]
1 20	2 20	3 20	4 20	5 20	

In [3]:
np.shape(ecg)

(100, 1025)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 20	2 20	3 20	4 20	5 20	
 max = 20


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (20, 1025)
Up to class  2 train shape =  (40, 1025)
Up to class  3 train shape =  (60, 1025)
Up to class  4 train shape =  (80, 1025)
Up to class  5 train shape =  (100, 1025)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0 20 20 20 20 20]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5., 5., 5., 5., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5., 5.])

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  3.2147954000000003
min magnitude class 1  =  -2.8853939
after normalizing >>
max magnitude class 1  =  1.1640359935145925
min magnitude class 1  =  -0.9968888607072098

 cls 2  >> 
max magnitude class 2  =  2.1563458
min magnitude class 2  =  -3.1616234000000003
after normalizing >>
max magnitude class 2  =  0.7890918835701477
min magnitude class 2  =  -1.0947401178836489

 cls 3  >> 
max magnitude class 3  =  3.2339357999999994
min magnitude class 3  =  -2.679459
after normalizing >>
max magnitude class 3  =  1.1708162693463189
min magnitude class 3  =  -0.9239386911507206

 cls 4  >> 
max magnitude class 4  =  2.9995496
min magnitude class 4  =  -3.2768523000000007
after normalizing >>
max magnitude class 4  =  1.0877875402105524
min magnitude class 4  =  -1.1355586862409686

 cls 5  >> 
max magnitude class 5  =  2.7769078
min magnitude class 5  =  -2.6748398
after normalizing >>
max magnitude class 5  =  1.0089191353876594
min magnitude class 5

In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -1.1355586862409686
max tst = 1.1708162693463189


In [14]:
len(wndws_test)

2425

In [15]:
np.shape(wndws_test)

(2425, 1025)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(20, 1025)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (100, 1024)
y_train =>  (100,)
X_test  =>  (2425, 1024)
y_test  =>  (2425,)
X_valid  =>  (100, 1024)
y_valid  =>  (100,)


In [22]:
if min(y_train)==0:
    print(' class was started from 0=Zero ')
else:
    print('!!!!!! class Should start from 0=Zero !!!!!!')
    print("!!!!!!   Reduce 1 From Class Number    !!!!!")
    y_train=np.array(y_train-1)
    y_test=np.array(y_test-1)
    y_valid=np.array(y_valid-1)
print('final labels=',set(y_train))

!!!!!! class Should start from 0=Zero !!!!!!
!!!!!!   Reduce 1 From Class Number    !!!!!
final labels= {0, 1, 2, 3, 4}


In [23]:
y_train_cat=to_categorical(y_train)#[:,1:]
y_valid_cat=to_categorical(y_valid)#[:,1:]
y_test_cat=to_categorical(y_test)#[:,1:]


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

<h1><center><div style="direction:rtl;font-family:B Nazanin">Network</div></center></h1>

In [24]:
# Import ResNet1D Module
#from ResNet_1DCNN import ResNet
#from ResNeXt_1DCNN import ResNeXt as ResNet
from SE_ResNeXt_1DCNN import SEResNeXt as ResNet

"Configurations for ResNet in Regression Mode"
length = X_train.shape[1]   # Number of Features (or length of the signal)
model_width = 128           # Number of Filter or Kernel in the Input Layer
num_channel = 1             # Number of Input Channels
problem_type = 'Classification' # Regression or Classification
output_number = cls_num     # Number of Outputs in the Regression Mode
Regression_Model = ResNet(length, num_channel, model_width, problem_type=problem_type, output_nums=output_number).SEResNeXt50() # Build Model
# SEResNext Models supported: SEResNeXt18, SEResNeXt34, SEResNeXt50, SEResNeXt101, SEResNeXt152, 
Regression_Model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 100, restore_best_weights = True)
history = Regression_Model.fit(X_train, y_train_cat, epochs=500, batch_size=btch, verbose=1, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping]) #,validation_split=0.2,callbacks =[callbacks]

Epoch 1/500
5/5 [==============================] - 57s 3s/step - loss: 3.4491 - accuracy: 0.3100 - val_loss: 3620062.5000 - val_accuracy: 0.2000
Epoch 2/500
5/5 [==============================] - 8s 2s/step - loss: 1.9854 - accuracy: 0.2900 - val_loss: 2156763.7500 - val_accuracy: 0.2000
Epoch 3/500
5/5 [==============================] - 6s 2s/step - loss: 1.8069 - accuracy: 0.4200 - val_loss: 599648.8125 - val_accuracy: 0.2000
Epoch 4/500
5/5 [==============================] - 6s 998ms/step - loss: 1.4308 - accuracy: 0.4200 - val_loss: 155077.6875 - val_accuracy: 0.2000
Epoch 5/500
5/5 [==============================] - 8s 2s/step - loss: 1.4050 - accuracy: 0.3900 - val_loss: 92153.9219 - val_accuracy: 0.2000
Epoch 6/500
5/5 [==============================] - 7s 2s/step - loss: 1.2575 - accuracy: 0.5200 - val_loss: 39986.5312 - val_accuracy: 0.2000
Epoch 7/500
5/5 [==============================] - 9s 2s/step - loss: 0.8215 - accuracy: 0.6800 - val_loss: 11687.6396 - val_accuracy: 0.2

Epoch 59/500
5/5 [==============================] - 6s 1s/step - loss: 0.3492 - accuracy: 0.8700 - val_loss: 9381.7930 - val_accuracy: 0.2000
Epoch 60/500
5/5 [==============================] - 6s 1s/step - loss: 0.3531 - accuracy: 0.8500 - val_loss: 1028.6791 - val_accuracy: 0.2000
Epoch 61/500
5/5 [==============================] - 8s 1s/step - loss: 0.2363 - accuracy: 0.9200 - val_loss: 4145.8545 - val_accuracy: 0.2000
Epoch 62/500
5/5 [==============================] - 6s 1s/step - loss: 0.2796 - accuracy: 0.8800 - val_loss: 3056.5549 - val_accuracy: 0.2000
Epoch 63/500
5/5 [==============================] - 8s 2s/step - loss: 0.3267 - accuracy: 0.9100 - val_loss: 3426.1387 - val_accuracy: 0.2000
Epoch 64/500
5/5 [==============================] - 6s 2s/step - loss: 0.1656 - accuracy: 0.9300 - val_loss: 1403.8164 - val_accuracy: 0.2000
Epoch 65/500
5/5 [==============================] - 6s 2s/step - loss: 0.2596 - accuracy: 0.9100 - val_loss: 4438.4204 - val_accuracy: 0.2000
Epoch 

Epoch 117/500
5/5 [==============================] - 7s 2s/step - loss: 0.0574 - accuracy: 0.9700 - val_loss: 15.4010 - val_accuracy: 0.5200
Epoch 118/500
5/5 [==============================] - 9s 2s/step - loss: 0.3754 - accuracy: 0.9300 - val_loss: 4.8021 - val_accuracy: 0.6900
Epoch 119/500
5/5 [==============================] - 6s 2s/step - loss: 0.0837 - accuracy: 0.9600 - val_loss: 4.1603 - val_accuracy: 0.6900
Epoch 120/500
5/5 [==============================] - 9s 2s/step - loss: 0.0219 - accuracy: 1.0000 - val_loss: 5.4318 - val_accuracy: 0.6800
Epoch 121/500
5/5 [==============================] - 6s 1s/step - loss: 0.2285 - accuracy: 0.9000 - val_loss: 41.1935 - val_accuracy: 0.2100
Epoch 122/500
5/5 [==============================] - 8s 2s/step - loss: 0.0577 - accuracy: 0.9900 - val_loss: 18.3112 - val_accuracy: 0.2700
Epoch 123/500
5/5 [==============================] - 6s 2s/step - loss: 0.0078 - accuracy: 1.0000 - val_loss: 12.6178 - val_accuracy: 0.4300
Epoch 124/500
5/

5/5 [==============================] - 9s 2s/step - loss: 0.0583 - accuracy: 0.9800 - val_loss: 0.0699 - val_accuracy: 0.9900
Epoch 176/500
5/5 [==============================] - 9s 2s/step - loss: 0.2111 - accuracy: 0.9800 - val_loss: 10.1070 - val_accuracy: 0.5800
Epoch 177/500
5/5 [==============================] - 7s 2s/step - loss: 0.1550 - accuracy: 0.9600 - val_loss: 32.9012 - val_accuracy: 0.4200
Epoch 178/500
5/5 [==============================] - 6s 2s/step - loss: 0.0187 - accuracy: 1.0000 - val_loss: 15.4932 - val_accuracy: 0.4900
Epoch 179/500
5/5 [==============================] - 7s 2s/step - loss: 0.2518 - accuracy: 0.9400 - val_loss: 12.6894 - val_accuracy: 0.4700
Epoch 180/500
5/5 [==============================] - 6s 2s/step - loss: 0.0226 - accuracy: 0.9900 - val_loss: 6.4184 - val_accuracy: 0.6000
Epoch 181/500
5/5 [==============================] - 9s 2s/step - loss: 0.0146 - accuracy: 1.0000 - val_loss: 2.9691 - val_accuracy: 0.7000
Epoch 182/500
5/5 [===========

5/5 [==============================] - 12s 2s/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 1.2844 - val_accuracy: 0.7600
Epoch 234/500
5/5 [==============================] - 9s 2s/step - loss: 0.1447 - accuracy: 0.9600 - val_loss: 2.2078 - val_accuracy: 0.7400
Epoch 235/500
5/5 [==============================] - 10s 2s/step - loss: 0.0569 - accuracy: 0.9800 - val_loss: 1.2794 - val_accuracy: 0.7900
Epoch 236/500
5/5 [==============================] - 7s 2s/step - loss: 0.1416 - accuracy: 0.9400 - val_loss: 8.1847 - val_accuracy: 0.5000
Epoch 237/500
5/5 [==============================] - 9s 2s/step - loss: 0.0088 - accuracy: 1.0000 - val_loss: 4.1489 - val_accuracy: 0.5700
Epoch 238/500
5/5 [==============================] - 6s 2s/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 2.2099 - val_accuracy: 0.6800
Epoch 239/500
5/5 [==============================] - 6s 2s/step - loss: 0.0078 - accuracy: 1.0000 - val_loss: 1.3550 - val_accuracy: 0.8300
Epoch 240/500
5/5 [=============

5/5 [==============================] - 8s 2s/step - loss: 0.0043 - accuracy: 1.0000 - val_loss: 0.1848 - val_accuracy: 0.9500
Epoch 292/500
5/5 [==============================] - 6s 1s/step - loss: 0.0862 - accuracy: 0.9800 - val_loss: 2.0043 - val_accuracy: 0.6800
Epoch 293/500
5/5 [==============================] - 8s 2s/step - loss: 0.0303 - accuracy: 0.9900 - val_loss: 0.4273 - val_accuracy: 0.8700
Epoch 294/500
5/5 [==============================] - 6s 2s/step - loss: 0.2258 - accuracy: 0.9500 - val_loss: 2.5939 - val_accuracy: 0.8400
Epoch 295/500
5/5 [==============================] - 9s 2s/step - loss: 0.0164 - accuracy: 0.9900 - val_loss: 14.8195 - val_accuracy: 0.5200
Epoch 296/500
5/5 [==============================] - 7s 2s/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 8.6820 - val_accuracy: 0.6200
Epoch 297/500
5/5 [==============================] - 6s 2s/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 6.7284 - val_accuracy: 0.6500
Epoch 298/500
5/5 [==============

In [25]:
predict_x=Regression_Model.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                 # function from augment.py to remove 0 index predictions

#y_pred_crs = model_Res.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

76/76 [==============================] - 26s 282ms/step
array([[410,   1,  19,  17,  11],
       [ 46, 576,  26,   5,   1],
       [ 11,   0, 385,  17,   5],
       [119,   6,  54, 386,  16],
       [ 17,   0,   4,   0, 293]], dtype=int64)
              precision    recall  f1-score   support

           0       0.68      0.90      0.77       458
           1       0.99      0.88      0.93       654
           2       0.79      0.92      0.85       418
           3       0.91      0.66      0.77       581
           4       0.90      0.93      0.92       314

    accuracy                           0.85      2425
   macro avg       0.85      0.86      0.85      2425
weighted avg       0.86      0.85      0.85      2425

